In [1]:
# set workspace
WhoIsWorking = "AA"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = r'C:\Users\achan\OneDrive\Desktop\Term Project Data'
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

C:\Users\achan\OneDrive\Desktop\Term Project Data


In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the data
neighbourhood_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\Neighbourhood_Cleaned.csv")
crime_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\MCI_Cleaned.csv")

# Merge the two datasets on the neighbourhood column
merged_data = pd.merge(neighbourhood_data, crime_data, left_on='NeighbourhoodName', right_on='NEIGHBOURHOOD_158')

# Select the columns to use for clustering
columns = ['Poles_Count', 'POI_Count', 'TCamera_Count', 'TotalPopulation', 'MedianTotalIncome', 'AverageTotalIncome', 'BuildingCoveragePERCENTAGE', 'ParksOSNAPERCENTAGE', 'BuiltUpPERCENTAGE', 'AveragePrice', 'OCC_YEAR', 'OCC_HOUR', 'TEMP']
data_for_clustering = merged_data[columns]

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_for_clustering)

# Perform K-means clustering
kmeans = KMeans(n_clusters=5, random_state=0)  # Change the number of clusters as needed
clusters = kmeans.fit_predict(data_scaled)

# Add the cluster labels to the original data
merged_data['Cluster'] = clusters

# Identify the high-risk areas
high_risk_areas = merged_data[merged_data['Cluster'] == merged_data['Cluster'].value_counts().idxmin()]

print(high_risk_areas)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values